In [1]:
# from module.generate_random_point import *
# import pandas as pd
# import numpy as np
# import warnings 

# warnings.filterwarnings('ignore')

# random_point = create_random_point_based_on_place("여의도동 서울특별시 대한민국", 430)

# driver = random_point[:30]
# item_O = random_point[30:230]
# item_D = random_point[230:]

# driver = pd.DataFrame(driver, columns=['geometry'])
# item = pd.DataFrame(item_O, columns=['O_point'])
# item['D_point'] = item_D

# driver['driver_id'] = range(len(driver))
# driver['work_start'] = 1020
# driver['work_end'] = 1200
# driver['current_lon'] = [i.x for i in driver['geometry']]
# driver['current_lat'] = [i.y for i in driver['geometry']]
# driver = driver.drop('geometry', axis=1)

# driver = driver[['driver_id', 'current_lat', 'current_lon', 'work_start', 'work_end']]


# item['O_lat'] = [i.y for i in item['O_point']]
# item['O_lon'] = [i.x for i in item['O_point']]
# item['D_lat'] = [i.y for i in item['D_point']]
# item['D_lon'] = [i.x for i in item['D_point']]
# item = item.drop(['O_point', 'D_point'], axis=1)

# item['ID'] = range(len(item))
# item['O_id'] = list(range(0, len(item)*2, 2))
# item['D_id'] = list(range(1, len(item)*2, 2))

# item['order_time'] = np.random.choice(range(1020, 1201),size=len(item))
# item = item[['ID', 'O_id', 'D_id', 'order_time', 'O_lat', 'O_lon', 'D_lat', 'D_lon']]

# item.to_pickle('item.pkl')
# driver.to_pickle('driver.pkl')

---
### **00. Library**

In [2]:
# library
import pandas as pd
import numpy as np
import warnings 
from tqdm import tqdm
import time
from IPython.display import clear_output 
import json
import itertools

warnings.filterwarnings('ignore')

# module
from module.osrm_api import *
from module.utils import *
from module.dispatch import *
from module.draw_base_map import *

---
### **01. Input Data prepared**

In [3]:
### 데이터 전처리
def input_data_prepared(driver, items):
    '''
    driver
    '''
    # driver 현재 위치 (초기 위치)
    driver['current_lat'] = [i.y for i in driver['geometry']]
    driver['current_lon'] = [i.x for i in driver['geometry']]
    # driver 고유 ID 지정
    driver['driver_id'] = range(len(driver))
    # 출퇴근 시간 지정
    driver['work_start'] = 1020
    driver['work_end'] = 1200
    # geometry
    driver = driver.drop('geometry', axis=1)
    # 정렬
    driver = driver[['driver_id', 'current_lat', 'current_lon', 'work_start', 'work_end']]
    
    '''
    items
    '''
    items['ID'] = range(len(items))

    items['O_id'] = list(range(0, len(items)*2, 2))
    items['D_id'] = list(range(1, len(items)*2, 2))

    items = items[['ID', 'O_id', 'D_id', 'ride_dtime', 'O_y', 'O_x', 'D_y', 'D_x']]
    items.columns = ['ID', 'O_id', 'D_id', 'order_time', 'O_lat', 'O_lon', 'D_lat', 'D_lon']
    
    return driver, items

---

### **02-00. Dispatch Helper function**

- **dispatch_data_prepared**
> - dispatch input 형식으로 만드는 함수

In [4]:
def dispatch_data_prepared(driver_obj_X, driver_obj_O, non_pickup_items, pickup_complete_items):
    
    driver_obj_X = driver_obj_X.reset_index(drop=True)
    driver_obj_O = driver_obj_O.reset_index(drop=True)
    non_pickup_items = non_pickup_items.reset_index(drop=True)
    
    D_ID_key = list(itertools.chain(*driver_obj_O['current_pickup_item_D_ID'].values.tolist()))
    pickup_complete_items = user_sort_values(pickup_complete_items, 'D_id', D_ID_key)
        
    '''
    coordinates
    '''
    driver_obj_X_coords = driver_obj_X[['current_lat', 'current_lon']].values.tolist()
    driver_obj_O_coords = driver_obj_O[['current_lat', 'current_lon']].values.tolist()
    O_items_coords = non_pickup_items[['O_lat', 'O_lon']].values.tolist()
    D_items_coords = non_pickup_items[['D_lat', 'D_lon']].values.tolist()
    delivery_coords = pickup_complete_items[['D_lat', 'D_lon']].values.tolist()
    
    coords = driver_obj_X_coords + driver_obj_O_coords + O_items_coords + D_items_coords + delivery_coords


    '''
    driver_info
    '''
    # driver_obj_X_info
    driver_obj_X_info = [{'driver_id':int(row['driver_id']), 'location_index': int(idx)} for idx,row in driver_obj_X.iterrows()]
    
    # driver_obj_O_info
    driver_obj_O_info = []

    count = 0
    
    for idx, row in driver_obj_O.iterrows():
        driver_obj_O_info_dict = dict()
        
        driver_obj_O_info_dict['driver_id'] = int(row['driver_id'])
        driver_obj_O_info_dict['location_index'] = int(idx+len(driver_obj_X))

        B_info_sub = []
        
        for id in row['current_pickup_item_D_ID']:
            B_info_sub_dict = dict()
            
            B_info_sub_dict['id'] = int(id)
            B_info_sub_dict['location_index'] = int(count+len(coords)-len(delivery_coords))
            B_info_sub.append(B_info_sub_dict)

            count += 1 

        driver_obj_O_info_dict['deliveries'] = B_info_sub
        driver_obj_O_info.append(driver_obj_O_info_dict)
        
    driver_info =  driver_obj_O_info + driver_obj_X_info


    '''
    items_info
    '''
    items_info =[{'pickup':{'id': int(row['O_id']), 'location_index': int(len(driver_info)+idx)},\
        'delivery':{'id': int(row['D_id']), 'location_index': int(len(driver_info)+len(non_pickup_items)+idx)}} for idx, row in non_pickup_items.iterrows()]


    '''
    dispatch_data
    '''
    dispatch_data = {
        'coordinates':coords,
        'drivers': driver_info,
        'shipments':items_info
    }
    
    return dispatch_data

### **02-01. Main Helper function** 

- **start_driver_prepared funtion**
> - 운행 시작하는 driver filter 
> - 사용하는 column 추가

In [5]:
def start_driver_prepared(data, time):
    data = data.loc[data['work_start'] == time]
    
    ### extra column
    # moving 움직이고 있는 driver check column
    data['moving'] = 0 

    ### 배송해야 되는 물품
    # ID (배송 순의 물품 O_ID, D_ID)
    data['delivery_OD_ID_list'] = [[] for _ in range(len(data))]
    # 물품 별 도착 예정 시간 
    data['delivery_OD_time_list'] = [[] for _ in range(len(data))]    
    # 현재 가지고 있는 item의 D ID
    data['current_pickup_item_D_ID'] = [[] for _ in range(len(data))]
    # 배송 순 route
    data['delivery_route_list'] = [[] for _ in range(len(data))]
    # 배송 순 timestamp 
    data['delivery_timestamp_list'] = [[] for _ in range(len(data))]
    # target route
    data['delivery_target_route'] = [[] for _ in range(len(data))]
    # target timestamp
    data['delivery_target_timestamp'] = [[] for _ in range(len(data))]

    
    ### 시뮬레이션 누적 데이터 
    ## trip 
    data['complete_route'] = [[] for _ in range(len(data))]
    data['complete_timestamp'] = [[] for _ in range(len(data))]
    
    
    ## point
    # (임시 데이터)
    data['temporary_point_geometry'] = [[] for _ in range(len(data))]
    data['temporary_point_time'] = [[] for _ in range(len(data))]
    
    # (저장 데이터)
    data['accumulate_point_geometry'] = [[] for _ in range(len(data))]
    data['accumulate_point_time'] = [[] for _ in range(len(data))]
     
    return data

- **order_shipment_prepared**
> - 주문한 shipment filter 
> - 사용하는 column 추가

In [6]:
def order_shipment_prepared(data, time=0):    
    data = data.loc[data['order_time'] == time]
    
    # pick-up과 delivery 여부 확인
    data['pickup_check'] = 0
    data['delivery_check'] = 0
    
    # pick-up한 시간과 delivery한 시간 
    data['pickup_time'] = 0
    data['delivery_time'] = 0
    
    # 물건 배달 driver id
    data['delivery_driver_id'] = 0
    
    return data    

- **transform_osrm_input_form**
> - osrm input 형식에 맞게 데이터 변환 (O_lat, O_lon, D_lat, D_lon)

In [7]:
def transform_osrm_input_form(coords_list):
    
    osrm_input_data = np.hstack([np.array(coords_list)[:-1], np.array(coords_list)[1:]])
    osrm_input_data = osrm_input_data.tolist()
    
    return osrm_input_data

- **timestamp_time_update**
> - 기존의 osrm_api의 timestamp output은 [0, 1, 1.32, 2.1, 3]  ->  current_time(ex. 360)을 더해주어서, [360, 361, 361.32, 362.1, 363]
> - 부수적으로 pickup 또는 delivery시 소요시간을 delay_time으로 주어 추가할 수 있다. (default : delay_time=0)

In [8]:
def timestamp_time_update(timestamp, current_time, delay_time):
    if delay_time == 'random':
        # 0~2분 사이의 시간으로 delay_time 지정
        delay_time = np.random.randint(200, size=len(timestamp[1:])) / 100
    
        timestamp[0] = list(np.array(timestamp[0]) + current_time)
        
        for idx, _ in enumerate(timestamp[1:]):
            timestamp[idx+1] = list(np.array(timestamp[idx+1]) + (timestamp[idx][-1] + delay_time))

        return timestamp
    else:
        delay_time = delay_time

        timestamp[0] = list(np.array(timestamp[0]) + current_time)
        
        for idx, _ in enumerate(timestamp[1:]):
            timestamp[idx+1] = list(np.array(timestamp[idx+1]) + (timestamp[idx][-1] + delay_time))
        
        return timestamp

---

- **current_data_update**
> - working driver update
> - dispatch의 결과를 기반으로 driver에 할당된 item 값을 업데이트

> - 물건이 할당된 driver는 moving(움직이고 있는지) 여부(0: 정지/ 1: 이동) 이동으로 변경 **[moving column update]**

> - driver에게 할당된 item의 O_id(pick-up 장소의 고유 ID), D_id(delivery 장소의 고유 ID)를 이동하는 순서대로 기입 **[delivery_OD_ID_list column update]**

> - driver에 할당된 item의 추천 경로(route) 전체 기입 **[delivery_route_list column update]**
> - driver에 할당된 item의 추천 경로(route)의 걸리는 시간(timestamp) 전체 기입 **[delivery_timestamp_list column update]**
> - driver에게 할당된 item의 O_id 또는 D_id까지의 예상 도착 시간 기입 **[delivery_OD_time_list column update]**

> - driver에 할당된 item의 추천 경로(route) 중 첫 번째 경로 기입 **[delivery_target_route column update]**
> - driver에 할당된 item의 추천 걸리는 시간(timestamp) 중 첫 번째 경로 기입 **[delivery_target_timestamp column update]**

> - item이 할당된 driver는 해당 시간(current_time)에 simulation시 점을 없애기 위해 좌표와 시간을 저장
>> - (처음에 할당된 driver는 바로 저장해준다) **[accumulate_point_geometry, accumulate_point_time column update]**

> - item이 할당된 것이 없는 driver에 temporary_point_geometry, temporary_point_time가 없다면, 해당 시간과 좌표를 넣어주어 저장해준다. 
>> **[temporary_point_geometry, temporary_point_time column update]**

In [9]:
### use function (사용하는 함수)
# user_sort_values : 사용자가 원하는 컬럼의 값 순서대로 정렬
# transform_osrm_input_form : OSRM API function에 들어가는 형식으로 데이터 변환
# get_res, get_total_route, get_total_timestamp : OSRM API를 사용하여, route, timestamp를 생성할 때 사용하는 함수
# timestamp_time_update : 추출한 timestamp를 simulation시 정한 time을 적용하여, [360, 363, 365, ...]([6시00분, 6시03분, 6시05분, ...])로 적용
def current_data_update(dispatch_result, coordinates, working_driver, time, delay_time=0):
    
    start_time, current_time = time
    
    '''----- 01. dispatch result를 기반으로 배정받은 driver 배정받지 못한 driver 데이터 분리'''
    # item 배송을 배정받은 driver ID
    assigned_driverID = [sub['driver_id'] for sub in dispatch_result]

    # assigned_working_driver : item 배정받은 driver data
    # unassigned_working_driver : item 배정받지 못한 driver data
    assigned_working_driver = working_driver.loc[working_driver['driver_id'].isin(assigned_driverID)]
    unassigned_working_driver = working_driver.loc[~working_driver['driver_id'].isin(assigned_driverID)]

    assigned_working_driver = user_sort_values(assigned_working_driver, 'driver_id', assigned_driverID)
    unassigned_working_driver = unassigned_working_driver.reset_index(drop=True)
    
    
    '''----- 02. 물건이 할당된 driver는 moving(움직이고 있는지) 여부(0: 정지/ 1: 이동) 이동으로 변경'''
    assigned_working_driver['moving'] = 1    
    unassigned_working_driver['moving'] = 0
    
    
    '''----- 03. driver에게 할당된 item의 O_id(pick-up 장소의 고유 ID), D_id(delivery 장소의 고유 ID)를 이동하는 순서대로 기입'''
    # new_delivery_OD_ID_list : eg. [[O_id, D_id, ...], [D_id, D_id, ...]]
    new_delivery_OD_ID_list = [[step['id'] for step in result['steps'] if step['id'] != -1] for result in dispatch_result]
    assigned_working_driver['delivery_OD_ID_list'] = new_delivery_OD_ID_list


    '''----- 04. driver에 할당된 item(O, D)의 route, timestamp, time(O_id, D_id에 도착 시간)를 OSRM API를 이용하여 update'''
    # dispatch result을 이용하여, osrm input 형식에 맞게 데이터를 변환
    osrm_input_data_by_driver = [transform_osrm_input_form([coordinates[step['location_index']] for step in result['steps']][:-1]) \
        for result in dispatch_result]

    # OSRM API를 이용하여, route, timestamp 추출
    all_step_list = [[get_res(sub_coord) for sub_coord in coord] for coord in osrm_input_data_by_driver]
    total_route = [[get_total_route(step) for step in all_step] for all_step in all_step_list]
    total_timestamp = [[get_total_timestamp(step) for step in all_step] for all_step in all_step_list]

    # route, timestamp, time update
    assigned_working_driver['delivery_route_list'] = total_route
    assigned_working_driver['delivery_timestamp_list'] = [timestamp_time_update(timestamp, current_time, delay_time=delay_time) for timestamp in total_timestamp]
    assigned_working_driver['delivery_OD_time_list'] = [[sub_ts[-1] for sub_ts in timestamp] for timestamp in assigned_working_driver['delivery_timestamp_list']]


    '''----- 05. driver에 할당된 route, timestamp의 첫번째 route, timestamp update'''
    assigned_working_driver['delivery_target_route'] = [route[0] for route in assigned_working_driver['delivery_route_list']]
    assigned_working_driver['delivery_target_timestamp'] = [timestamp[0] for timestamp in assigned_working_driver['delivery_timestamp_list']]


    '''----- 06. driver의 moving에 따른 point, time 저장'''
    if start_time == current_time:
        ## assigned_working_driver
        accumulate_point_geometry_list = []
        accumulate_point_time_list = []

        for _, row in assigned_working_driver.iterrows():
            accumulate_point_geometry = row['accumulate_point_geometry']
            accumulate_point_time = row['accumulate_point_time']
            lat = row['current_lat']
            lon = row['current_lon']
            
            accumulate_point_geometry.append([lon,lat])
            accumulate_point_time.append([start_time, start_time])
            
            accumulate_point_geometry_list.append(accumulate_point_geometry)
            accumulate_point_time_list.append(accumulate_point_time)
            
        assigned_working_driver['accumulate_point_geometry'] = accumulate_point_geometry_list
        assigned_working_driver['accumulate_point_time'] = accumulate_point_time_list
        
        
        ## unassigned_working_driver
        temporary_point_geometry_list = []
        temporary_point_time_list = []

        for _, row in unassigned_working_driver.iterrows():
            temporary_point_geometry = row['temporary_point_geometry']
            temporary_point_time = row['temporary_point_time']
            lat = row['current_lat']
            lon = row['current_lon']
            
            temporary_point_geometry.extend([lon,lat])
            temporary_point_time.extend([start_time])
            
            temporary_point_geometry_list.append(temporary_point_geometry)
            temporary_point_time_list.append(temporary_point_time)
            
        unassigned_working_driver['temporary_point_geometry'] = temporary_point_geometry_list
        unassigned_working_driver['temporary_point_time'] = temporary_point_time_list    
    else:
        ## assigned_working_driver
        # temporary_data가 있으면 이제 움직이기 시작한 driver이기 때문에 temporary_data를 완성시켜 accumulate_point로 넘긴다.
        assigned_working_driver_A = assigned_working_driver.loc[[len(i)!=0 for i in assigned_working_driver['temporary_point_geometry']]]
        # temporary_point_geometry가 없으면 움직이던 시작한 driver이기 때문에 pass
        assigned_working_driver_B = assigned_working_driver.loc[[len(i)==0 for i in assigned_working_driver['temporary_point_geometry']]]
        
        assigned_working_driver_A = assigned_working_driver_A.reset_index(drop=True)
        assigned_working_driver_B = assigned_working_driver_B.reset_index(drop=True)
        
        accumulate_point_geometry_list = []
        accumulate_point_time_list = []

        for idx, row in assigned_working_driver_A.iterrows():
            accumulate_point_geometry = row['accumulate_point_geometry']
            accumulate_point_time = row['accumulate_point_time']
            
            stop_time = row['temporary_point_time']
            stop_time.extend([current_time])
            
            lon_lat = row['temporary_point_geometry']

            accumulate_point_geometry.append(lon_lat)
            accumulate_point_time.append(stop_time)
            
            accumulate_point_geometry_list.append(accumulate_point_geometry)
            accumulate_point_time_list.append(accumulate_point_time)
            
            # 입력 후 초기화
            assigned_working_driver_A['temporary_point_time'].iloc[idx] = []
            assigned_working_driver_A['temporary_point_geometry'].iloc[idx] = []
            
        assigned_working_driver_A['accumulate_point_geometry'] = accumulate_point_geometry_list
        assigned_working_driver_A['accumulate_point_time'] = accumulate_point_time_list

        assigned_working_driver = pd.concat([assigned_working_driver_A, assigned_working_driver_B])
        assigned_working_driver = assigned_working_driver.reset_index(drop=True)        


        ## unassigned_working_driver
        # temporary_data가 있으면 정지해 있던 driver이기 때문에 pass
        unassigned_working_driver_A = unassigned_working_driver.loc[[len(i)!=0 for i in unassigned_working_driver['temporary_point_geometry']]]
        # temporary_data가 움직이다 멈춤 driver이기 때문에 현재 geometry와 time를 update
        unassigned_working_driver_B = unassigned_working_driver.loc[[len(i)==0 for i in unassigned_working_driver['temporary_point_geometry']]]
        
        unassigned_working_driver_A = unassigned_working_driver_A.reset_index(drop=True)
        unassigned_working_driver_B = unassigned_working_driver_B.reset_index(drop=True)
        
        # unassigned_working_driver
        temporary_point_geometry_list = []
        temporary_point_time_list = []

        for _, row in unassigned_working_driver_B.iterrows():
            lat = row['current_lat']
            lon = row['current_lon']
            
            lon_lat = [lon, lat]
                        
            temporary_point_geometry_list.append(lon_lat)
            temporary_point_time_list.append([current_time])
            
        unassigned_working_driver_B['temporary_point_geometry'] = temporary_point_geometry_list
        unassigned_working_driver_B['temporary_point_time'] = temporary_point_time_list
        
        unassigned_working_driver = pd.concat([unassigned_working_driver_A, unassigned_working_driver_B])
        unassigned_working_driver = unassigned_working_driver.reset_index(drop=True) 
        
    return assigned_working_driver, unassigned_working_driver

- **nextTime_data_update**
> - next time(current_time + 1)이 되면 물건을 배정받은 driver의 pick-up & delivery를 update, 또한 움직이는 driver의 next time의 위치를 update
> - ass_working_driver의 'delivery_OD_time_list'를 확인하여, pick-up & delivery가 next time에 도착하는지 확인 후 처리
> - next time에 아직 움직이는 driver 데이터 처리 및 좌표 업데이트

In [10]:
# split_geometry_based_on_time : route, timestamp를 원하는 시간을 기준으로 쪼개는 함수
def nextTime_data_update(assigned_working_driver, order_item, delivery_item, current_time):
    
    next_time = current_time + 1
    
    complete_item = pd.DataFrame(columns=['ID', 'O_id', 'D_id', 'order_time', 'O_lat', 'O_lon', 'D_lat', 'D_lon',
                                          'pickup_check', 'delivery_check', 'pickup_time','delivery_time', 'delivery_driver_id'])
    
    '''01. next time에 pick-up & delivery location에 도착한 유무로 데이터 분리'''
    # target_working_driver : pick-up locations & delivery locations에 도착한 driver
    # non_target_working_driver : pick-up locations & delivery locations에 도착하지 않은 driver
    target_working_driver = assigned_working_driver.loc[[time[0] <= next_time  for time in assigned_working_driver['delivery_OD_time_list']]]
    non_target_working_driver = assigned_working_driver.loc[[not(time[0] <= next_time)  for time in assigned_working_driver['delivery_OD_time_list']]]
    
    target_working_driver = target_working_driver.reset_index(drop=True)
    non_target_working_driver = non_target_working_driver.reset_index(drop=True)
    
    '''02. next time에 pick-up & delivery location에 도착한 driver & item 데이터 처리'''
    if len(target_working_driver) > 0:
        row_list = []
        
        for idx in range(len(target_working_driver)):
            # data 한 행 
            row = target_working_driver.iloc[[idx]]
            
            while sum(np.array(row['delivery_OD_time_list'].iloc[0]) <= next_time):
                # first_dest_time :pick-up location & delivery location에 도착한 시작 eg. 1201                
                first_time = row['delivery_OD_time_list'].iloc[0][0].copy()
                # first_dest_state : OD ID가 짝수이면 'pick-up' location 홀수이면 'delivery' location로 정의
                first_state = 'pick-up' if row['delivery_OD_ID_list'].iloc[0][0] % 2 == 0 else 'delivery'
                # first__ID : item ID 
                first_ID =  int(row['delivery_OD_ID_list'].iloc[0][0] / 2) if first_state == 'pick-up' else \
                    int((row['delivery_OD_ID_list'].iloc[0][0]-1) / 2)
                
                #####################
                ## driver data update
                #####################
                
                ## OD_ID & OD_time remove (도착이 완료한 item O & D ID, time 제거)
                # 완료된 OD_ID 값 제거 
                row['delivery_OD_ID_list'].iloc[0] = row['delivery_OD_ID_list'].iloc[0][1:]
                # 완료된 OD_time 값 제거 
                row['delivery_OD_time_list'].iloc[0] = row['delivery_OD_time_list'].iloc[0][1:]
    
                ## complete route & timestamp update (도착이 완료된 route & timestamp 저장)
                save_target_route = row['delivery_route_list'].iloc[0][0]
                save_target_timestamp = row['delivery_timestamp_list'].iloc[0][0]

                complete_route = row['complete_route'].iloc[0]
                complete_timestamp = row['complete_timestamp'].iloc[0]

                complete_route.append(save_target_route)
                complete_timestamp.append(save_target_timestamp)

                row['complete_route'].iloc[0] = complete_route
                row['complete_timestamp'].iloc[0] = complete_timestamp
                
                ## 도착이 완료된 delivery_route_list & delivery_timestamp_list 제거 
                # 완료된 route 값 제거 
                row['delivery_route_list'].iloc[0] = row['delivery_route_list'].iloc[0][1:]
                # 완료된 timestamp 값 제거
                row['delivery_timestamp_list'].iloc[0] = row['delivery_timestamp_list'].iloc[0][1:]
                
                
                if first_state == 'pick-up':
                    #####################
                    ## driver data update
                    #####################
                    ## driver가 현재 가지고 있는 도착 장소 ID 저장
                    delivery_loc_ID = (first_ID * 2) + 1

                    current_pickup_item_ID = row['current_pickup_item_D_ID'].iloc[0]
                    current_pickup_item_ID.extend([delivery_loc_ID])
                    
                    
                    row['current_pickup_item_D_ID'].iloc[0] = current_pickup_item_ID

                    
                    ## target route & timestamp update
                    # target route update 
                    row['delivery_target_route'].iloc[0] = row['delivery_route_list'].iloc[0][0]
                    # target timestamp update 
                    row['delivery_target_timestamp'].iloc[0] = row['delivery_timestamp_list'].iloc[0][0]
                    
                    
                    ####################
                    ## items data update
                    ####################
                    # 물건의 pickup 여부 변경 
                    order_item.loc[order_item['ID'] == first_ID, 'pickup_check'] = 1
                    # 물건의 pickup 시간 저장
                    order_item.loc[order_item['ID'] == first_ID, 'pickup_time'] = first_time
                    # 물건을 배당 받은 driver ID 저장
                    order_item.loc[order_item['ID'] == first_ID, 'delivery_driver_id']  = row['driver_id'].iloc[0]

                    # pick up된 item data -> delivery data로 변경
                    new_delivery_item = order_item.loc[order_item['ID'] == first_ID]
                    delivery_item = pd.concat([delivery_item, new_delivery_item])
                    delivery_item = delivery_item.reset_index(drop=True)
                    
                    order_item = order_item.loc[order_item['ID'] != first_ID]
                    order_item = order_item.reset_index(drop=True)
                    
                elif first_state == 'delivery':
                    # driver가 현재 가지고 있던 도착 장소 ID 제거 (배달 완료)
                    delivery_loc_ID = (first_ID * 2) + 1

                    current_pickup_item_ID = row['current_pickup_item_D_ID'].iloc[0]
                    current_pickup_item_ID.remove(delivery_loc_ID) 

                    row['current_pickup_item_D_ID'].iloc[0] = current_pickup_item_ID

                    ####################
                    ## items data update
                    ####################        
                    # 물건의 delivery 여부 변경
                    delivery_item.loc[delivery_item['ID'] == first_ID, 'delivery_check'] = 1
                    # 물건의 delivery 시간 저장
                    delivery_item.loc[delivery_item['ID'] == first_ID, 'delivery_time'] = first_time
                    
                    # delivery된 delivery data -> complete data로 변경
                    new_comple_item = delivery_item.loc[delivery_item['ID'] == first_ID]
                    complete_item = pd.concat([complete_item, new_comple_item])
                    complete_item = complete_item.reset_index(drop=True)
                    
                    delivery_item = delivery_item.loc[delivery_item['ID'] != first_ID]
                    delivery_item = delivery_item.reset_index(drop=True)
                    
                    ### delivery이 마지막일때, 마지막이 아닐때 
                    last_step_check = len(row['delivery_OD_ID_list'].iloc[0])
                    
                    ## delivery가 마지막일때
                    if last_step_check == 0:
                        #####################
                        ## driver data update
                        #####################
                        ## target route & timestamp remove
                        # target route update 
                        row['delivery_target_route'].iloc[0] = []
                        # target timestamp update 
                        row['delivery_target_timestamp'].iloc[0] = []
                        ## 멈춰있는 driver로 변경
                        row['moving'].iloc[0] = 0
                        ## 멈춰있기 시작하는 좌표와 시간 저장
                        lon_lat = row['complete_route'].iloc[0][-1][-1]
                        target_time = first_time
                        row['temporary_point_geometry'].iloc[0] = lon_lat
                        row['temporary_point_time'].iloc[0] = [target_time]
                        ## current lat, lon update
                        row['current_lat'].iloc[0] = lon_lat[1]
                        row['current_lon'].iloc[0] = lon_lat[0]
                        
                    ## delivery가 마지막이 아닐때
                    elif last_step_check > 0:
                        #####################
                        ## driver data update
                        #####################
                        ## target route & timestamp update
                        # target route update 
                        row['delivery_target_route'].iloc[0] = row['delivery_route_list'].iloc[0][0]
                        # target timestamp update 
                        row['delivery_target_timestamp'].iloc[0] = row['delivery_timestamp_list'].iloc[0][0]
            
            # 처리된 행 저장
            row_list.append(row)
        
        # 처리된 행 병합
        target_working_driver = pd.concat(row_list)
        target_working_driver = target_working_driver.reset_index(drop=True)
    
    # target이 있는 데이터만 좌표를 update를 하기 위해 병합
    assigned_working_driver = pd.concat([target_working_driver, non_target_working_driver])
    assigned_working_driver = assigned_working_driver.reset_index(drop=True)
            
            
    '''03. next time에 pick-up & delivery location에 도착한 driver & item 데이터 처리'''
    # target_working_driver : 현재 이동 중이여서, 좌표 업데이트가 필요한 경우
    # non_target_working_driver : 배송을 완료하여, 좌표 업데이트가 필요없는 경우
    target_working_driver = assigned_working_driver.loc[assigned_working_driver['moving'] == 1] 
    non_target_working_driver = assigned_working_driver.loc[assigned_working_driver['moving'] == 0]

    if len(target_working_driver) > 0:
        row_list = []
        
        for idx in range(len(target_working_driver)):
            row = target_working_driver.iloc[[idx]]
            
            ### 새로운 위치 좌표로 업데이트
            new_route, _, new_timestamp, _, new_coord = split_geometry_based_on_time(row['delivery_target_route'].iloc[0], 
                                                                                     row['delivery_target_timestamp'].iloc[0], time=next_time)
            
            # 새로운 위도, 경도 업데이트
            row['current_lat'].iloc[0] = new_coord[0]
            row['current_lon'].iloc[0] = new_coord[1]
            
            # 1분간 이동한 route, timestamp 저장            
            complete_route = row['complete_route'].iloc[0]
            complete_timestamp = row['complete_timestamp'].iloc[0]
            
            complete_route.append(new_route)
            complete_timestamp.append(new_timestamp)
            
            row['complete_route'].iloc[0] = complete_route
            row['complete_timestamp'].iloc[0] = complete_timestamp
            
            # 1분간 이동한 데이터 update후 pickup한 item_ID한 빼고 초기화 (pick up를 다른 사람이 할 수 있기 때문)
            row['delivery_OD_ID_list'].iloc[0] = []
            row['delivery_OD_time_list'].iloc[0] = []

            row['delivery_route_list'].iloc[0] = []
            row['delivery_timestamp_list'].iloc[0] = []

            row['delivery_target_route'].iloc[0] = []
            row['delivery_target_timestamp'].iloc[0] = []
            
            row_list.append(row)
        
        # 좌표 업데이트 후 driver data 병합
        target_working_driver = pd.concat(row_list)    
        target_working_driver = target_working_driver.reset_index(drop=True)
    
    # 모든 처리 후 driver data 병합 
    assigned_working_driver = pd.concat([target_working_driver, non_target_working_driver])
    assigned_working_driver = assigned_working_driver.reset_index(drop=True)
    

    ### 반환 전 index 정리
    order_item = order_item.reset_index(drop=True)
    delivery_item = delivery_item.reset_index(drop=True)
    complete_item = complete_item.reset_index(drop=True)
    
    return assigned_working_driver, order_item, delivery_item, complete_item

- **main**

In [11]:
### Data Load
# driver = pd.read_pickle('./data/driver_initi_location.pkl')
# driver = pd.DataFrame(driver, columns=['geometry'])
# items = pd.read_pickle('./data/passenger_final.pkl')

# driver, items = input_data_prepared(driver, items)

# items = items.sample(n=200)
# items = items.reset_index(drop=True)

In [12]:
driver = pd.read_pickle('./data/driver.pkl')
items = pd.read_pickle('./data/items.pkl')

In [13]:
'''
MAIN
'''
### ----- 00. parameters
# simulation 시작, 끝 시간
start_time, end_time = 1020, 1440


### ----- 01. main code 돌릴 때, 필요한 변수 정의
## driver (운송수단)
# working_driver : 일하고 있는 driver 
working_driver = pd.DataFrame()

## items (배송해야되는 물건)
# delivery_complete_time : 배달이 완료된 items
delivery_complete_item = pd.DataFrame()
# order_items : 배송해야 되는 items
order_items = pd.DataFrame(columns=['ID', 'O_id', 'D_id', 'order_time', 'O_lat', 'O_lon', 'D_lat', 'D_lon',
                                    'pickup_check', 'delivery_check', 'pickup_time','delivery_time', 'delivery_driver_id'])
# delivery_items : pick-up이 완료되어 delivery만 하면 되는 items
delivery_items = pd.DataFrame(columns=['ID', 'O_id', 'D_id', 'order_time', 'O_lat', 'O_lon', 'D_lat', 'D_lon',
                                       'pickup_check', 'delivery_check', 'pickup_time','delivery_time', 'delivery_driver_id'])

### ----- 02. main code 
for current_time in tqdm(range(start_time, end_time)):

    '''----- 02-1. 운행 시작한 driver & 배달 요청한 items 가져오기 (current_time 때)'''
    # new_driver
    start_working_driver = start_driver_prepared(driver, current_time)
    # new_items
    start_order_items = order_shipment_prepared(items, current_time)
    
    
    '''----- 02-2. working_driver(운행 가능한 driver) & order_items(현재 배송해야되는 물건) update'''
    # 새로운 driver(운행 가능한 driver) update : working_driver에 추가 
    working_driver = pd.concat([working_driver, start_working_driver])
    working_driver = working_driver.reset_index(drop=True)
    # 새로운 items update : order_items에 추가
    order_items = pd.concat([order_items, start_order_items])
    order_items = order_items.reset_index(drop=True)
    # delivery_items : 해당 데이터는 마지막에 추가 (배송 배정 후 pick-up 된 물건 데이터 의미)
    
    if ((len(delivery_items) + len(order_items))!=0) & (len(working_driver)!=0): # driver나 item이 있을 때만 실행
        '''----- 02-3. dispatch'''
        ## dispatch driver는 pick-up한 물건 유무에 따라 데이터를 분리한다.
        # driver_A : pick-up한 물건이 없는 driver
        # driver_B : pick-up한 물건이 있는 driver
        driver_A = working_driver.loc[[len(item) == 0 for item in working_driver['current_pickup_item_D_ID'].values]]
        driver_B = working_driver.loc[[len(item) != 0 for item in working_driver['current_pickup_item_D_ID'].values]]
        
        ## dispatch input data 형식으로 변환
        # -input data-
        # driver_A : pick-up한 물건이 없는 driver
        # driver_B : pick-up한 물건이 있는 driver
        # order_items : pick-up 해야되는 items
        # delivery_items :  pick-up 후 delivery 해야되는 items 
        # -output data-
        # dispatch_input_data : dispatch input data 
        dispatch_input_data = dispatch_data_prepared(driver_A, driver_B, order_items, delivery_items)
        
        ## main dispatch (dispatch function)
        # -input data-
        # dispatch_input_data : dispatch input data 
        # -output data-
        # dispatch_result : dispatch 결과
        # coordinates : 좌표 데이터 
        dispatch_result, coordinates = dispatch_module(dispatch_input_data)
        
        
        '''----- 02-4. current time working_driver update'''
        # -input data-
        # dispatch_result : dispatch 결과
        # coordinates : 좌표 데이터 
        # working_driver : 일하고 있는 driver
        # current_time : simulation 시점에 해당 시간 
        # -output data-
        # ass_working_driver : current_time 시점에서 물건을 할당 받은 driver
        # unass_working_driver : current_time 시점에서 물건을 할당 받지 못한 driver
        ass_working_driver, unass_working_driver = current_data_update(dispatch_result, coordinates, working_driver, [start_time, current_time])

        '''----- 02-5. next time working_driver & items update''' 
        # -input data-
        # ass_working_driver : current_time 시점에서 물건을 할당 받은 driver
        # order_items : pick-up 해야되는 items
        # delivery_items : pick-up이 완료되어 delivery만 하면 되는 items
        # current_time : *
        # -output data-
        # ass_working_driver : *
        # order_items : *
        # delivery_items : * 
        # complete_item : 배송을 완료한 item
        ass_working_driver, order_items, delivery_items, complete_item = nextTime_data_update(ass_working_driver, order_items, delivery_items, current_time)
        
        
        '''----- 02-6. 모든 처리 후 데이터 처리'''
        # 배송 완료한 item 데이터 저장
        delivery_complete_item = pd.concat([delivery_complete_item, complete_item])
        delivery_complete_item = delivery_complete_item.reset_index(drop=True)

        # working driver 데이터 병합 (ass_working_driver, unass_working_driver)
        working_driver = pd.concat([ass_working_driver, unass_working_driver])
        working_driver = working_driver.reset_index(drop=True)
        
        '''----- 02-7. driver, item 시각화'''
        # print(dispatch_result)
        # base_map = draw_map(working_driver, order_items, delivery_items)
        # display(base_map)
        # time.sleep(1)
        # clear_output()
    

100%|██████████| 420/420 [01:16<00:00,  5.52it/s] 


- 처리 후 전처리 및 저장

In [14]:
accumulate_point_geometry_list = []
accumulate_point_time_list = []

for idx, row in working_driver.iterrows():
    accumulate_point_geometry = row['accumulate_point_geometry']
    accumulate_point_time = row['accumulate_point_time']
    
    stop_time = row['temporary_point_time']
    stop_time.extend([current_time])
    
    lon_lat = row['temporary_point_geometry']

    accumulate_point_geometry.append(lon_lat)
    accumulate_point_time.append(stop_time)
    
    accumulate_point_geometry_list.append(accumulate_point_geometry)
    accumulate_point_time_list.append(accumulate_point_time)
    
    # 입력 후 초기화
    working_driver['temporary_point_time'].iloc[idx] = []
    working_driver['temporary_point_geometry'].iloc[idx] = []
    
working_driver['accumulate_point_geometry'] = accumulate_point_geometry_list
working_driver['accumulate_point_time'] = accumulate_point_time_list

In [18]:
# driver trip simulation data
all_trip = []

working_driver_A = working_driver.loc[[len(i) != 0 for i in working_driver['complete_route']]]
working_driver_B = working_driver.loc[[len(i) == 0 for i in working_driver['complete_route']]]
working_driver_A = working_driver_A.reset_index(drop=True)
working_driver_B = working_driver_B.reset_index(drop=True)

for index, row in working_driver_A.iterrows():
    
    for idx in range(len(row['complete_route'])):
        trip = [{'driver_id':row['driver_id'] ,'trip':working_driver_A['complete_route'].iloc[index][idx], 'timestamp':working_driver_A['complete_timestamp'].iloc[index][idx]}\
            for idx in range(len(working_driver_A['complete_route'].iloc[index]))]
        all_trip.extend(trip)

with open(f'./result_data/trip.json', 'w') as f:
    json.dump(all_trip, f)
        
# driver point simulation data
all_point_trip = []

for index, row in working_driver.iterrows():
    
    for idx in range(len(row['accumulate_point_geometry'])):
        trip = [{'driver_id':row['driver_id'] ,'location':working_driver['accumulate_point_geometry'].iloc[index][idx], 'timestamp':working_driver['accumulate_point_time'].iloc[index][idx]}\
            for idx in range(len(working_driver['accumulate_point_geometry'].iloc[index]))]
        all_point_trip.extend(trip)
        
with open(f'./result_data/driver_point.json', 'w') as f:
    json.dump(all_point_trip, f)
    
# item point simulatoin data        
all_item_O_trip = []
all_item_D_trip = []

for _, row in delivery_complete_item.iterrows():
    all_item_O_trip.append({'Item_ID': row['ID'],'location':[row['O_lon'], row['O_lat']], 'timestamp':[row['order_time'], row['pickup_time']]})
    all_item_D_trip.append({'Item_ID': row['ID'],'location':[row['D_lon'], row['D_lat']], 'timestamp':[row['pickup_time'], row['delivery_time']]})
    
with open(f'./result_data/item_O_point.json', 'w') as f:
    json.dump(all_item_O_trip, f)
    
with open(f'./result_data/item_D_point.json', 'w') as f:
    json.dump(all_item_D_trip, f)

In [ ]:
# import math
# import json
# import os

# os.chdir('c:/Users/user/Desktop/')

# with open('trip.json', 'r') as f:
#     data = json.load(f)

# for i in range(1020, 1441):
#     res = list(filter(lambda x: math.floor(x['timestamp'][0]) == i, data))
#     if len(res) == 0:
#         with open(f'./data/trip_{i}.json', 'w') as f:
#             json.dump([], f)
#     else:
#         with open(f'./data/trip_{i}.json', 'w') as f:
#             json.dump(res, f)